In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Titanic: Machine Learning from Disaster - End-to-End Pipeline
# ------------------------------------------------------------
# Covers: Data loading, preprocessing, pipeline building, cross-validation, and model comparison.

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# ==============================
# Load Dataset
# ==============================
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

# Target and Features
X = train.drop(columns=["Survived", "PassengerId", "Name", "Ticket", "Cabin"])
y = train["Survived"]

# ==============================
# Preprocessing
# ==============================
# Separate categorical & numerical features
categorical = X.select_dtypes(include=["object"]).columns.tolist()
numerical = X.select_dtypes(exclude=["object"]).columns.tolist()

# Numerical pipeline: Impute missing values + scale
num_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Categorical pipeline: Impute + OneHotEncode
cat_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, numerical),
        ("cat", cat_transformer, categorical)
    ]
)

# ==============================
# Models to Compare
# ==============================
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=200, random_state=42, use_label_encoder=False, eval_metric="logloss")
}

# ==============================
# Cross-validation Scores
# ==============================
for name, model in models.items():
    clf = Pipeline(steps=[("preprocessor", preprocessor),
                          ("classifier", model)])
    scores = cross_val_score(clf, X, y, cv=5, scoring="accuracy")
    print(f"{name}: Mean CV Accuracy = {scores.mean():.4f} (+/- {scores.std():.4f})")

# ==============================
# Train Best Model (XGBoost example)
# ==============================
best_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", XGBClassifier(n_estimators=200, random_state=42, use_label_encoder=False, eval_metric="logloss"))
])

best_model.fit(X, y)

# Predict on test set
X_test = test.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])
predictions = best_model.predict(X_test)

# ==============================
# Submission File
# ==============================
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predictions
})
submission.to_csv("submission.csv", index=False)
print("✅ Submission file created: submission.csv")


Logistic Regression: Mean CV Accuracy = 0.7879 (+/- 0.0198)
Random Forest: Mean CV Accuracy = 0.8048 (+/- 0.0291)
XGBoost: Mean CV Accuracy = 0.8070 (+/- 0.0255)
✅ Submission file created: submission.csv
